Key Metrics & Signals (the “+1 / -1 / Above / Under” stuff)
The paper uses a scoring system for stock-picking, where variables are assigned +1 (positive signal), -1 (negative signal), or 0 (neutral). Main factors include:
--
Earnings Momentum
EPS growth > 25% YoY → +1
EPS growth negative → -1
--
Valuation Multiples
P/E ratio < 15 and PEG < 1 → +1
P/E > 40 → -1
--
Return Ratios
ROE > 20% → +1
ROE < 10% → -1
--
Debt Levels
Debt-to-Equity < 0.5 → +1
Debt-to-Equity > 1.5 → -1
--
Price Action / Momentum
Price above 200-day moving average → +1
Price below 200-day moving average → -1
--
Insider Activity (EDGAR)
Net insider buying → +1
Net insider selling → -1
--


In [19]:
# GS13VDSMCEAZQL7C

In [20]:
import pandas as pd
import numpy as np
import requests
import time
from IPython.display import display, Markdown

def fetch_alpha_vantage_data(function, symbol, api_key):
    """
    Fetches data from the Alpha Vantage API for a given function and symbol.

    This function constructs the correct API URL, makes the request, and
    handles potential errors from the network or the API itself. It is
    designed to be used for fundamental data functions like OVERVIEW,
    INCOME_STATEMENT, BALANCE_SHEET, and CASH_FLOW.

    Args:
        function (str): The Alpha Vantage API function to call (e.g., 'OVERVIEW').
        symbol (str): The stock ticker symbol (e.g., 'AAPL').
        api_key (str): Your personal Alpha Vantage API key.

    Returns:
        dict: The JSON data returned by the API as a Python dictionary.

    Raises:
        Exception: If the API request fails, the response is invalid, or the
                   API returns an error message.
    """
    # The base URL for the Alpha Vantage API.
    base_url = "https://www.alphavantage.co/query?"
    
    # Construct the full API URL using the provided arguments.
    # The data is returned in JSON format by default.
    url = (
        f"{base_url}function={function}&symbol={symbol}"
        f"&apikey={api_key}"
    )

    try:
        # Make the GET request to the API.
        response = requests.get(url)
        
        # Raise an HTTPError for bad responses (4xx or 5xx status codes).
        response.raise_for_status()

        # Parse the JSON response.
        data = response.json()

        # Check for specific error messages from the Alpha Vantage API itself.
        if 'Error Message' in data:
            raise Exception(f"Alpha Vantage API error: {data['Error Message']}")
        if 'Information' in data and 'Thank you for using Alpha Vantage' in data['Information']:
            print("Note: API call frequency limit reached. Waiting and retrying...")
            # Implement a simple delay if the user hits the rate limit.
            # This is a basic retry mechanism.
            time.sleep(16) # Alpha Vantage free tier is 5 calls per minute (12 seconds per call)
            return fetch_alpha_vantage_data(function, symbol, api_key)

        return data

    except requests.exceptions.RequestException as e:
        # Catch any network-related errors.
        raise Exception(f"Network request failed: {e}")
    except ValueError as e:
        # Catch JSON decoding errors.
        raise Exception(f"Failed to decode JSON from response: {e}")

# Symbols to analyze (top 5 NASDAQ by market cap; static list for simplicity)
symbols = ['AAPL','MSFT','NVDA','AMZN','GOOGL']

# Reuse existing helper
fetch = fetch_alpha_vantage_data
API_KEY = '1VWXIAX2LM6F6WV0'

company_data = {}
for sym in symbols:
    try:
        ov = fetch('OVERVIEW', sym, API_KEY)
        is_ = fetch('INCOME_STATEMENT', sym, API_KEY)
        bs = fetch('BALANCE_SHEET', sym, API_KEY)
        cf = fetch('CASH_FLOW', sym, API_KEY)
        company_data[sym] = {'overview': ov, 'income': is_, 'bs': bs, 'cf': cf}
    except Exception as e:
        company_data[sym] = {'error': str(e)}

# Display all the downloaded data for each company
display(Markdown("### Downloaded Data Overview"))
for sym, data in company_data.items():
    display(Markdown(f"#### Data for {sym}"))
    if 'error' in data:
        display(Markdown(f"**Error:** {data['error']}"))
    else:
        display(Markdown(f"Keys available: {', '.join(data.keys())}"))
        for key, value in data.items():
            display(Markdown(f"**{key.capitalize()} Data:**"))
            display(value)


### Downloaded Data Overview

#### Data for AAPL

Keys available: overview, income, bs, cf

**Overview Data:**

{'Symbol': 'AAPL',
 'AssetType': 'Common Stock',
 'Name': 'Apple Inc',
 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.",
 'CIK': '320193',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Industry': 'ELECTRONIC COMPUTERS',
 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US',
 'OfficialSite': 'https://www.apple.com',
 'FiscalYearEnd': 'September',
 'LatestQuarter': '2025-06-30',
 'MarketCapitalization': '3403645714000',
 'EBITDA': '141696008000',
 'PERatio

...

{'symbol': 'GOOGL',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'grossProfit': '203712000000',
   'totalRevenue': '350018000000',
   'costOfRevenue': '146306000000',
   'costofGoodsAndServicesSold': '146306000000',
   'operatingIncome': '112390000000',
   'sellingGeneralAndAdministrative': '14188000000',
   'researchAndDevelopment': '49326000000',
   'operatingExpenses': '91322000000',
   'investmentIncomeNet': 'None',
   'netInterestIncome': '4214000000',
   'interestIncome': '4482000000',
   'interestExpense': '268000000',
   'nonInterestIncome': 'None',
   'otherNonOperatingIncome': 'None',
   'depreciation': 'None',
   'depreciationAndAmortization': '15311000000',
   'incomeBeforeTax': '119815000000',
   'incomeTaxExpense': '19697000000',
   'interestAndDebtExpense': 'None',
   'netIncomeFromContinuingOperations': '100118000000',
   'comprehensiveIncomeNetOfTax': 'None',
   'ebit': '120083000000',
   'ebitda': '135394000000',
   'netIncome

**Bs Data:**

{'symbol': 'GOOGL',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'totalAssets': '450256000000',
   'totalCurrentAssets': '163711000000',
   'cashAndCashEquivalentsAtCarryingValue': '23466000000',
   'cashAndShortTermInvestments': '23466000000',
   'inventory': 'None',
   'currentNetReceivables': '52340000000',
   'totalNonCurrentAssets': '286545000000',
   'propertyPlantEquipment': 'None',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'intangibleAssets': 'None',
   'intangibleAssetsExcludingGoodwill': 'None',
   'goodwill': '31885000000',
   'investments': 'None',
   'longTermInvestments': '37982000000',
   'shortTermInvestments': '72191000000',
   'otherCurrentAssets': '15714000000',
   'otherNonCurrentAssets': 'None',
   'totalLiabilities': '125172000000',
   'totalCurrentLiabilities': '89122000000',
   'currentAccountsPayable': '7987000000',
   'deferredRevenue': 'None',
   'currentDebt': 'None',
   'shortTermDebt': '2887000000',
 

**Cf Data:**

{'symbol': 'GOOGL',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'operatingCashflow': '125299000000',
   'paymentsForOperatingActivities': 'None',
   'proceedsFromOperatingActivities': 'None',
   'changeInOperatingLiabilities': 'None',
   'changeInOperatingAssets': 'None',
   'depreciationDepletionAndAmortization': '15311000000',
   'capitalExpenditures': '52535000000',
   'changeInReceivables': 'None',
   'changeInInventory': 'None',
   'profitLoss': 'None',
   'cashflowFromInvestment': '-45536000000',
   'cashflowFromFinancing': '-79733000000',
   'proceedsFromRepaymentsOfShortTermDebt': 'None',
   'paymentsForRepurchaseOfCommonStock': 'None',
   'paymentsForRepurchaseOfEquity': 'None',
   'paymentsForRepurchaseOfPreferredStock': 'None',
   'dividendPayout': '7363000000',
   'dividendPayoutCommonStock': '7363000000',
   'dividendPayoutPreferredStock': 'None',
   'proceedsFromIssuanceOfCommonStock': 'None',
   'proceedsFromIssuanceOfLongTermDe

In [24]:
# Build one-row per company with ratios (using latest annual reports)

def to_float(x):
    try:
        return float(x)
    except Exception:
        return np.nan

rows = []
for sym in symbols:
    data = company_data.get(sym, {})
    ov = data.get('overview', {}) if isinstance(data.get('overview', {}), dict) else {}
    is_ann = (data.get('income', {}) or {}).get('annualReports', [])
    bs_ann = (data.get('bs', {}) or {}).get('annualReports', [])
    cf_ann = (data.get('cf', {}) or {}).get('annualReports', [])

    is0 = is_ann[0] if len(is_ann) > 0 else {}
    bs0 = bs_ann[0] if len(bs_ann) > 0 else {}

    # Core numbers
    revenue = to_float(is0.get('totalRevenue'))
    gross_profit = to_float(is0.get('grossProfit'))
    operating_income = to_float(is0.get('operatingIncome'))
    ebit = to_float(is0.get('ebit')) if is0.get('ebit') not in (None, 'None', '') else to_float(is0.get('operatingIncome'))
    net_income = to_float(is0.get('netIncome'))
    interest_expense = to_float(is0.get('interestExpense')) if is0.get('interestExpense') not in (None, 'None', '') else to_float(is0.get('interestAndDebtExpense'))

    cur_assets = to_float(bs0.get('totalCurrentAssets'))
    cur_liab = to_float(bs0.get('totalCurrentLiabilities'))
    inventory = to_float(bs0.get('inventory'))
    cash_eq = to_float(bs0.get('cashAndCashEquivalentsAtCarryingValue'))
    if np.isnan(cash_eq):
        cash_eq = to_float(bs0.get('cashAndShortTermInvestments'))
    total_assets = to_float(bs0.get('totalAssets'))
    total_liab = to_float(bs0.get('totalLiabilities'))
    equity = to_float(bs0.get('totalShareholderEquity'))

    total_debt = to_float(bs0.get('shortLongTermDebtTotal'))
    if np.isnan(total_debt):
        total_debt = to_float(bs0.get('longTermDebt')) + to_float(bs0.get('shortTermDebt'))

    # Market
    price_per_share = np.nan
    if ov.get('MarketCapitalization') and ov.get('SharesOutstanding'):
        price_per_share = to_float(ov['MarketCapitalization']) / max(to_float(ov['SharesOutstanding']), 1.0)
    eps_ttm = to_float(ov.get('DilutedEPSTTM') or ov.get('EPS'))
    book_value_per_share = to_float(ov.get('BookValue'))

    # Liquidity
    current_ratio = cur_assets / cur_liab if cur_liab and cur_liab != 0 else np.nan
    quick_ratio = (cur_assets - inventory) / cur_liab if cur_liab and cur_liab != 0 else np.nan
    cash_ratio = cash_eq / cur_liab if cur_liab and cur_liab != 0 else np.nan

    # Profitability
    gross_margin = (gross_profit / revenue * 100) if revenue else np.nan
    op_margin = (operating_income / revenue * 100) if revenue else np.nan
    net_margin = (net_income / revenue * 100) if revenue else np.nan
    roa = (net_income / total_assets * 100) if total_assets else np.nan
    roe = (net_income / equity * 100) if equity else np.nan
    roce = (ebit / (total_assets - cur_liab) * 100) if (total_assets and cur_liab and (total_assets - cur_liab) != 0) else np.nan

    # Solvency
    debt_to_equity = (total_debt / equity) if equity else np.nan
    equity_ratio = (equity / total_assets * 100) if total_assets else np.nan
    interest_cov = (ebit / interest_expense) if interest_expense not in (None, 0, np.nan) else np.nan
    debt_ratio = (total_liab / total_assets) if total_assets else np.nan

    # Efficiency (annual; averages not available cross-year here, so mark N/A)
    inv_turn = np.nan
    ar_turn = np.nan
    ap_turn = np.nan
    asset_turn = (revenue / total_assets) if total_assets else np.nan
    wc_turn = np.nan

    # Market ratios
    pe_ratio = (price_per_share / eps_ttm) if eps_ttm not in (None, 0, np.nan) else np.nan
    pb_ratio = (price_per_share / book_value_per_share) if book_value_per_share not in (None, 0, np.nan) else np.nan

    rows.append({
        'symbol': sym,
        # Liquidity
        'current_ratio': current_ratio,
        'quick_ratio': quick_ratio,
        'cash_ratio': cash_ratio,
        # Profitability
        'gross_profit_margin_pct': gross_margin,
        'operating_profit_margin_pct': op_margin,
        'net_profit_margin_pct': net_margin,
        'roa_pct': roa,
        'roe_pct': roe,
        'roce_pct': roce,
        # Solvency
        'debt_to_equity': debt_to_equity,
        'equity_ratio_pct': equity_ratio,
        'interest_coverage': interest_cov,
        'debt_ratio': debt_ratio,
        # Efficiency
        'inventory_turnover': inv_turn,
        'ar_turnover': ar_turn,
        'ap_turnover': ap_turn,
        'asset_turnover': asset_turn,
        'working_capital_turnover': wc_turn,
        # Market
        'pe_ratio': pe_ratio,
        'pb_ratio': pb_ratio
    })

ratios_table = pd.DataFrame(rows).set_index('symbol')

# Display, formatting
with pd.option_context('display.float_format', lambda v: f"{v:,.2f}"):
    display(Markdown('### Company Ratios (latest annual)'))
    display(ratios_table)

# Note for missing efficiency ratios
display(Markdown("_Note: Some efficiency ratios left N/A due to lack of average quarterly values across years._"))


### Company Ratios (latest annual)

,current_ratio,quick_ratio,cash_ratio,gross_profit_margin_pct,operating_profit_margin_pct,net_profit_margin_pct,roa_pct,roe_pct,roce_pct,debt_to_equity,equity_ratio_pct,interest_coverage,debt_ratio,inventory_turnover,ar_turnover,ap_turnover,asset_turnover,working_capital_turnover,pe_ratio,pb_ratio
symbol,,,,,,,,,,,,,,,,,,,,
AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MSFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NVDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


_Note: Some efficiency ratios left N/A due to lack of average quarterly values across years._